# Introduction to NLP Fundamentals in TensorFlow

## Check for GPU

In [56]:
!nvidia-smi -L


/bin/bash: line 1: nvidia-smi: command not found


## Get helper functions

In [57]:
!wget https://raw.githubusercontent.com/yacin-hamdi/deep_learning/master/tensorflow/helper_functions.py

--2024-02-13 17:42:16--  https://raw.githubusercontent.com/yacin-hamdi/deep_learning/master/tensorflow/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-02-13 17:42:17 (82.0 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [58]:
from helper_functions import *

## Get a text dataset

In [59]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2024-02-13 17:42:17--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.214.207, 172.253.114.207, 108.177.111.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.214.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2024-02-13 17:42:17 (110 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



In [60]:
unzip_data("nlp_getting_started.zip")

## Visualizing a text dataset

In [61]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [62]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [63]:
# shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [64]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [65]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [66]:
# total number of samples
len(train_df), len(test_df)

(7613, 3263)

In [67]:
# visualize some random training examples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f'target:{target}', "(real disaster)" if target > 0 else "(not real disater)")
  print(f'test:\n{text}')
  print("----\n")

target:1 (real disaster)
test:
Woman's GPS app guides rescuers to injured biker in Marin County - SFGate http://t.co/Iz9U8BhfAA
----

target:0 (not real disater)
test:
Has an accident changed your life? We will help you determine options that can financially support life care plans and on-going treatment.
----

target:0 (not real disater)
test:
If you told me ten years ago that I'd be seeing anime on the big screen... ...I probably would have exploded
----

target:1 (real disaster)
test:
The RCMP are reporting fatalities and serious injuries as a result of the collision on the TCH near Whitbourne.
----

target:1 (real disaster)
test:
MEG issues Hazardous Weather Outlook (HWO)  http://t.co/gGn39m60tL #WX
----



### Split data into training and validation sets

In [68]:
from sklearn.model_selection import train_test_split

In [69]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                            train_df_shuffled['target'].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=32)

In [70]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [71]:
# Chekc the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['#LOL Plymouth (\x89Û÷Let\x89Ûªs Obliterate Litter\x89Ûª) http://t.co/GDrssjbH8q',
        'AND MY FAM HAD TO EVACUATE BC WE NEED POWER',
        'MH370: Aircraft debris found on La Reunion is from missing Malaysia Airlines ... - ABC Onlin... http://t.co/N3lNdJKYo3 G #Malaysia #News',
        '\x89Û÷Good Samaritans\x89Ûª shot in horror hijacking http://t.co/V5yUUALoqw #263Chat #Twimbos ZimpapersViews',
        "#FOXDebateQuestions:  To what degree has Obama's efforts to institute Sharia Law exacerbated the California wild fires?",
        "Looks like a war zone outside. What's going on?",
        '#hot  Funtenna: hijacking computers to send data as sound waves [Black Hat 2015] http://t.co/8JcYXhq1AZ #prebreak #best',
        "Doing Giveaway Music Kit Dren Death's Head Demolition: http://t.co/fHKhCqPl7j",
        'BBC News - India rail crash: Trains derail in Madhya Pradesh flash flood http://t.co/fU1Btuq1Et',
        "'Gunman who opened fire at Tennessee movie theater killed by

## Converting text into number

 ### Text Vectorization

In [72]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [73]:
text_vectorizer = TextVectorization(max_tokens=1000,
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode='int',
                                    output_sequence_length=None,
                                    pad_to_max_tokens=True)

In [74]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [75]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=max_length)

In [76]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [77]:
# Create a sample sentence and tokenize it
sample_sentence = "there's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[266,   3, 208,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [78]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f'original text:\n{random_sentence}\
        \n\nVectorized version:')
text_vectorizer([random_sentence])

original text:
@missleylaha I didn't get to buy one after the last London show because the fire alarm went off and everyone had to be evacuated. hahahaha        

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1,    8,  555,   49,    5, 1145,   59,   37,    2,  149, 1115,
         337,  156,    2,   42]])>

In [79]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()

In [80]:
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
top_5_words, bottom_5_words

(['', '[UNK]', 'the', 'a', 'in'],
 ['pantofel', 'panties', 'panther', 'pantalonesfuego', 'panoramic'])

### Creating an Embedding using an Embedding Layer

In [81]:
import tensorflow as tf

embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length=max_length)

embedding

In [82]:
random_sentence = random.choice(train_sentences)
print(f'original text:\n {random_sentence}\
      \n\nEmbedding version:')
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

original text:
 The spread of Conflict #PalmOil has sparked an increase in fires destroying #forest throughout #Indonesia http://t.co/4zn0MDsRVp      

Embedding version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01593168,  0.03442253, -0.04785054, ..., -0.01872182,
          0.01008172,  0.04209436],
        [-0.00985502,  0.0078151 , -0.04898926, ..., -0.01216307,
         -0.00336009,  0.03938612],
        [ 0.02838573,  0.00232144,  0.01076059, ..., -0.00385641,
          0.03826157, -0.03848056],
        ...,
        [ 0.03616288,  0.02699392,  0.0186622 , ...,  0.00915132,
         -0.01391413, -0.04121614],
        [-0.00012703,  0.02339855,  0.04721384, ...,  0.0401857 ,
         -0.00349652, -0.01410396],
        [ 0.03610022, -0.0187626 ,  0.02428308, ...,  0.02065459,
          0.00393987,  0.02843586]]], dtype=float32)>

In [83]:
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 1.59316771e-02,  3.44225280e-02, -4.78505380e-02,  2.75682919e-02,
         2.50541233e-02,  1.13025308e-05, -1.90180074e-02, -1.90369375e-02,
         4.36605699e-02,  1.56562217e-02,  3.86155285e-02, -2.91159749e-02,
         4.20731641e-02,  1.39391311e-02,  2.52785943e-02,  4.11567949e-02,
        -4.56480496e-02, -3.47163677e-02, -8.85231420e-03, -4.66452353e-02,
         2.26825364e-02, -2.12510359e-02,  7.45851919e-03,  3.42899077e-02,
         3.53470780e-02,  4.13028859e-02, -1.27596371e-02, -4.99791168e-02,
        -3.23088653e-02, -3.81001458e-02,  3.19501273e-02,  4.68670167e-02,
         2.83977427e-02, -4.52008620e-02, -3.43633443e-02, -4.28841226e-02,
        -2.95365341e-02,  4.37514894e-02,  3.83501537e-02,  1.86048187e-02,
        -3.61949094e-02, -3.51539031e-02, -8.45824555e-03,  4.66096140e-02,
         3.64375748e-02, -8.89196247e-03,  2.47142464e-03,  4.90356795e-02,
         1.92971863e-02,  2.20757723e-0

## Modelling a text dataset

* Model 0: Naive Bayes (baseline)
* Model 1: feed-forward neural network (dense model)
* Model 2: LSTM model(RNN)
* Model 3: GRU model(RNN)
* Model 4: bidirectional-LSTM model(RNN)
* Model 5: 1D Convolutional Neural Network (CNN)
* Model 6: Tensorflow Hub Pretrained Feature Extrator
* Model 7: Same as model 6 with 10% of training data

### Model 0: baseline

In [84]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ('clf', MultinomialNB())
])

# Fit the pipline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [85]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f'our baseline model achieves an accuracy of:{baseline_score*100:.2f}%')

our baseline model achieves an accuracy of:79.53%


In [86]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [87]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1])

### evaluation function

In [88]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def evaluations(y_true, y_pred):
  eval = {}
  eval['accuracy'] = accuracy_score(y_true, y_pred) * 100
  eval['precision'] = precision_score(y_true, y_pred, average='weighted')
  eval['recall'] = recall_score(y_true, y_pred, average='weighted')
  eval['f1'] = f1_score(y_true, y_pred, average='weighted')

  return eval



In [89]:
baseline_results = evaluations(val_labels, baseline_preds)
baseline_results

{'accuracy': 79.52755905511812,
 'precision': 0.8123798624937205,
 'recall': 0.7952755905511811,
 'f1': 0.7894130279169649}

### Model 1: simple dense model

In [90]:
# Build model with the functional API
inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_1 = tf.keras.Model(inputs, outputs, name='model_1_dense')

In [91]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_1  (None, 128)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [92]:
model_1.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [93]:
history_1 = model_1.fit(x=train_sentences,
                        y=train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 9s 37ms/step - loss: 0.6125 - accuracy: 0.6859 - val_loss: 0.5235 - val_accuracy: 0.7638
Epoch 2/5
215/215 [==============================] - 7s 34ms/step - loss: 0.4437 - accuracy: 0.8175 - val_loss: 0.4539 - val_accuracy: 0.7953
Epoch 3/5
215/215 [==============================] - 8s 39ms/step - loss: 0.3481 - accuracy: 0.8615 - val_loss: 0.4450 - val_accuracy: 0.8071
Epoch 4/5
215/215 [==============================] - 7s 35ms/step - loss: 0.2854 - accuracy: 0.8895 - val_loss: 0.4533 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 8s 38ms/step - loss: 0.2387 - accuracy: 0.9132 - val_loss: 0.4694 - val_accuracy: 0.7953


In [94]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 11ms/step - loss: 0.4694 - accuracy: 0.7953


[0.46935275197029114, 0.7952755689620972]

In [95]:
model_1_predictions = model_1.predict(val_sentences)

24/24 [==============================] - 0s 7ms/step


In [96]:
model_1_predictions.shape

(762, 1)

In [97]:
model_1_predictions[:10]

array([[0.27787253],
       [0.15676026],
       [0.02468986],
       [0.9247484 ],
       [0.00399424],
       [0.7366899 ],
       [0.00928276],
       [0.4045154 ],
       [0.18872961],
       [0.36328515]], dtype=float32)

In [98]:
model_1_preds = tf.squeeze(tf.round(model_1_predictions))

In [99]:
model_1_results = evaluations(val_labels, model_1_preds)

In [100]:
model_1_results

{'accuracy': 79.52755905511812,
 'precision': 0.7973088191970407,
 'recall': 0.7952755905511811,
 'f1': 0.7935657758060216}

## Visualizing learned embeddings

In [101]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [102]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_1  (None, 128)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 128

In [104]:
# Get the weight matrix of embeding layer
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
embed_weights.shape

(10000, 128)

In [105]:
# Create embedding files
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [ ]:
# Download files from colab projector.tensorflow.org
# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass

## Recurrent Neural Networks (RNN)

## Model 2: LSTM

In [106]:
inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = tf.keras.layers.LSTM(units=64, return_sequences=True)(x)
x = tf.keras.layers.LSTM(64)(x)
# x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")


In [107]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm_3 (LSTM)               (None, 64)                49408     
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1329473 (5.07 MB)
Trainable params: 1329473 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [108]:
model_2.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [109]:
history_2 = model_2.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 14s 51ms/step - loss: 0.2235 - accuracy: 0.9202 - val_loss: 0.5810 - val_accuracy: 0.7835
Epoch 2/5
215/215 [==============================] - 10s 47ms/step - loss: 0.1552 - accuracy: 0.9403 - val_loss: 0.5584 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 10s 47ms/step - loss: 0.1268 - accuracy: 0.9542 - val_loss: 0.6397 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 12s 57ms/step - loss: 0.1014 - accuracy: 0.9639 - val_loss: 0.7855 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 9s 40ms/step - loss: 0.0852 - accuracy: 0.9654 - val_loss: 0.9340 - val_accuracy: 0.7651


In [110]:
model_2_pred_probs = model_2.predict(val_sentences)


24/24 [==============================] - 1s 6ms/step


In [111]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))

In [112]:
model_2_results = evaluations(val_labels, model_2_preds)

In [113]:
model_2_results

{'accuracy': 76.50918635170603,
 'precision': 0.7660746138711035,
 'recall': 0.7650918635170604,
 'f1': 0.7633165022184397}

In [115]:
inputs = tf.keras.layers.Input(shape=(1, ), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
x = tf.keras.layers.LSTM(64)(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model_2_1 = tf.keras.Model(inputs, outputs)
model_2_1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_3 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm_6 (LSTM)               (None, 15, 64)            49408     
                                                                 
 lstm_7 (LSTM)               (None, 64)                33024     
                                                                 
 dense_8 (Dense)             (None, 64)                4160      
                                                           

In [116]:
model_2_1.compile(loss='binary_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])

In [117]:
history_2_1 = model_2_1.fit(train_sentences,
                            train_labels,
                            epochs=5,
                            validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 20s 55ms/step - loss: 0.1413 - accuracy: 0.9552 - val_loss: 0.7916 - val_accuracy: 0.7664
Epoch 2/5
215/215 [==============================] - 10s 44ms/step - loss: 0.0733 - accuracy: 0.9717 - val_loss: 1.0101 - val_accuracy: 0.7625
Epoch 3/5
215/215 [==============================] - 14s 65ms/step - loss: 0.0604 - accuracy: 0.9734 - val_loss: 1.1971 - val_accuracy: 0.7467
Epoch 4/5
215/215 [==============================] - 18s 85ms/step - loss: 0.0511 - accuracy: 0.9771 - val_loss: 1.4218 - val_accuracy: 0.7664
Epoch 5/5
215/215 [==============================] - 19s 87ms/step - loss: 0.0473 - accuracy: 0.9775 - val_loss: 1.4662 - val_accuracy: 0.7638


### Model 3: GRU

In [125]:
inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = tf.keras.layers.GRU(64)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs)
model_3.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_3 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru_3 (GRU)                 (None, 64)                37248     
                                                                 
 dense_13 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1317313 (5.03 MB)
Trainable params: 1317313 (5.03 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________

In [126]:
model_3.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [127]:
history_3 = model_3.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels))

Epoch 1/5
215/215 [==============================] - 14s 51ms/step - loss: 0.1241 - accuracy: 0.9518 - val_loss: 1.1170 - val_accuracy: 0.7559
Epoch 2/5
215/215 [==============================] - 13s 59ms/step - loss: 0.0550 - accuracy: 0.9769 - val_loss: 1.1035 - val_accuracy: 0.7612
Epoch 3/5
215/215 [==============================] - 10s 45ms/step - loss: 0.0437 - accuracy: 0.9810 - val_loss: 1.6251 - val_accuracy: 0.7454
Epoch 4/5
215/215 [==============================] - 12s 57ms/step - loss: 0.0403 - accuracy: 0.9807 - val_loss: 1.3003 - val_accuracy: 0.7533
Epoch 5/5
215/215 [==============================] - 12s 57ms/step - loss: 0.0377 - accuracy: 0.9815 - val_loss: 1.6232 - val_accuracy: 0.7520


In [128]:
model_3_pred_probs = model_3.predict(val_sentences)


24/24 [==============================] - 1s 9ms/step


In [129]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))

In [131]:
model_3_results = evaluations(val_labels, model_3_preds)
model_3_results

{'accuracy': 75.19685039370079,
 'precision': 0.7515322581711256,
 'recall': 0.7519685039370079,
 'f1': 0.7514568121059509}